In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:

from autotst.species import Conformer

#from autotst.molecule import AutoTST_Molecule
#from ase.io import write
import ase
from ase.calculators.gaussian import *
#from autotst.calculators.gaussian import AutoTST_Gaussian
import subprocess
import shlex
from subprocess import call
from shlex import split
from ase.io.gaussian import read_gaussian_out
from rmgpy.molecule import Molecule as RMG_Molecule

In [9]:
#subprocess.call(shlex.split('sbatch rotors_run_template.sh {0}'.format(geo_filename)))

In [45]:
from autotst.species import Species as TS_Species
from autotst.species import Conformer
from rmgpy.species import Species as RMG_Species
from rmgpy.molecule import Molecule as RMG_Molecule
from rdkit import Chem
import os
import arkane

class fake_statmech():
    
    def __init__(self):
        self.reactants = []
        self.products = []
        self.ts = None
        
        self.model_chemistry = 'M06-2X/cc-pVTZ'
        self.rotors = []
        self.ark_gaus = None
        return
    
    
    def add_rotor(self, conformer, torsion):
        mol = conformer.rmg_molecule
        
        a = min(torsion.atom_indices[1:3])
        b = max(torsion.atom_indices[1:3])
        tor_center = [a,b]
        
        tor_log = mol.toAugmentedInChIKey() + '_tor{0}{1}.log'.format(a,b)
        self.ark_gaus = arkane.gaussian.GaussianLog(tor_log)
        
        top_IDs = self.get_top_IDs(mol, tor_center)
        self.rotors[tuple(tor_center)] = "     HinderedRotor(scanLog=Log('{0}'), pivots={1}, top={2}, fit='fourier'),".format(tor_log, tor_center, top_IDs)
        return
    
    def get_top_IDs(self, conformer, tor_center):
        mol = conformer.rmg_molecule
        
        assert len(tor_center) == 2
        assert isinstance(tor_center, list)
        
        tor_bond = None
        for bond in mol.getAllEdges():
            a = bond.atom1.id
            b = bond.atom2.id
            if (tor_center == [a,b]) or (tor_center == [b,a]):
                tor_bond = bond
                break
        assert tor_bond is not None

        mol.removeBond(tor_bond)
        tops = mol.split()
        assert len(tops) == 2
        
        return [atom.id for atom in tops[0].atoms]
        
    def write_arkane_for_reacts_and_prods(self, conf, label, path=None):
        """
        a method to write species to an arkane input file. Mol is an RMGMolecule
        """
        #conf = Conformer(species.smiles[0])
        
        if path is None:
            path = os.getcwd()
            
        mol = conf.rmg_molecule
        freq_log = label + '_Freq'

        output = ['#!/usr/bin/env python',
                  '# -*- coding: utf-8 -*-', '', 'atoms = {']

        atom_dict = self.get_atoms(species)

        for atom, count in atom_dict.iteritems():
            output.append("    '{0}': {1},".format(atom, count))
        output = output + ['}', '']

        bond_dict = self.get_bonds(species)
        if bond_dict != {}:
            output.append('bonds = {')
            for bond_type, num in bond_dict.iteritems():
                output.append("    '{0}': {1},".format(bond_type, num))
            output.append("}")
        else:
            output.append('bonds = {}')

        #label = Chem.rdinchi.InchiToInchiKey(Chem.MolToInchi(Chem.MolFromSmiles(mol.toSMILES()))).strip("-N")

        external_symmetry = mol.getSymmetryNumber()

        output += ["", "linear = False", "", "externalSymmetry = {}".format(external_symmetry), "",
                   "spinMultiplicity = {}".format(mol.multiplicity), "", "opticalIsomers = 1", ""]

        output += ["energy = {", "    '{0}': Log('{1}.log'),".format(
            self.model_chemistry, freq_log), "}", ""]

        output += ["geometry = Log('{0}.log')".format(freq_log), ""]

        output += ["frequencies = Log('{0}.log')".format(freq_log), ""]

        output += ["rotors = ["]
        for rotor_info in self.rotors:
            output += rotor_info
        output += ["]"]

        #input_string = ""

        #for t in output:
        #    #input_string += t + "\n"
        
        input_string = '\n'.join(output)

        filename = label + '.py'
        with open(os.path.join(path, filename), "w") as f:
            f.write(input_string)
        
        return input_string
            
    def get_atoms(self, conf):
        """
        A method to create an atom dictionary for an rmg molecule
        """
        atom_dict = {}

        #conf = Conformer(species.smiles[0])

        rmg_mol = conf.rmg_molecule

        for atom in rmg_mol.atoms:
            if atom.isCarbon():
                atom_type = "C"
            if atom.isHydrogen():
                atom_type = "H"
            if atom.isOxygen():
                atom_type = "O"

            try:
                atom_dict[atom_type] += 1
            except KeyError:
                atom_dict[atom_type] = 1

        return atom_dict

    def get_bonds(self, conf):


        #conf = Conformer(species.smiles[0])
        
        rmg_mol = conf.rmg_molecule

        bondList = []
        for atom in rmg_mol.atoms:
            for bond in atom.bonds.values():
                bondList.append(bond)
        bonds = list(set(bondList))
        bondDict = {}
        for bond in bonds:
            if bond.isSingle():
                if bond.atom1.symbol == 'C' and bond.atom2.symbol == 'C':
                    bondType = 'C-C'
                elif (bond.atom1.symbol == 'H' and bond.atom2.symbol == 'H'):
                    bondType = 'H-H'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'H') or (bond.atom1.symbol == 'H' and bond.atom2.symbol == 'C'):
                    bondType = 'C-H'
                elif (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'O'):
                    bondType = 'O-O'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'O') or (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'C'):
                    bondType = 'C-O'
                elif (bond.atom1.symbol == 'H' and bond.atom2.symbol == 'O') or (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'H'):
                    bondType = 'O-H'
                elif bond.atom1.symbol == 'N' and bond.atom2.symbol == 'N':
                    bondType = 'N-N'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'C'):
                    bondType = 'N-C'
                elif (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'O'):
                    bondType = 'N-O'
                elif (bond.atom1.symbol == 'H' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'H'):
                    bondType = 'N-H'
                elif bond.atom1.symbol == 'S' and bond.atom2.symbol == 'S':
                    bondType = 'S-S'
                elif (bond.atom1.symbol == 'H' and bond.atom2.symbol == 'S') or (bond.atom1.symbol == 'S' and bond.atom2.symbol == 'H'):
                    bondType = 'S-H'
            elif bond.isDouble:
                if bond.atom1.symbol == 'C' and bond.atom2.symbol == 'C':
                    bondType = 'C=C'
                elif (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'O'):
                    bondType = 'O=O'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'O') or (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'C'):
                    bondType = 'C=O'
                elif bond.atom1.symbol == 'N' and bond.atom2.symbol == 'N':
                    bondType = 'N=N'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'C'):
                    bondType = 'N=C'
                elif (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'O'):
                    bondType = 'N=O'
                elif (bond.atom1.symbol == 'O' and bond.atom2.symbol == 'S') or (bond.atom1.symbol == 'S' and bond.atom2.symbol == 'O'):
                    bondType = 'S=O'
            elif bond.isTriple:
                if bond.atom1.symbol == 'C' and bond.atom2.symbol == 'C':
                    bondType = 'C#C'
                elif bond.atom1.symbol == 'N' and bond.atom2.symbol == 'N':
                    bondType = 'N#N'
                elif (bond.atom1.symbol == 'C' and bond.atom2.symbol == 'N') or (bond.atom1.symbol == 'N' and bond.atom2.symbol == 'C'):
                    bondType = 'N#C'
            try:
                bondDict[bondType] += 1
            except KeyError:
                bondDict[bondType] = 1

        return bondDict

In [11]:
"""x = RMG_Molecule()
SML = 'CC(CO)(C)OC'
x.fromSMILES(SML)
piv_bond = None
i = 1
for atom in x.atoms:
    atom.id = i
    i+=1

for bond in x.getAllEdges():
    print bond.atom1.id, bond.atom2.id
    if (bond.atom1.id == 2 and bond.atom2.id == 4) or (bond.atom1.id == 2 and bond.atom2.id == 4):
        piv_bond = bond
print "Piv Bond: ", piv_bond"""
x=1

In [12]:
from autotst.species import Species as TS_Species

SML = 'CC(CO)(C)OC'
aspec = TS_Species(smiles=[SML])
xmol = aspec.conformers.values()[0][0].rmg_molecule
#print xmol.toAugmentedInChIKey()
i = 1
for atom in xmol.atoms:
    atom.id = i
    i += 1

for bond in xmol.getAllEdges():
    x = bond.atom1
    y = bond.atom2
    
    if x.element.symbol == 'C' and y.element.symbol == 'C':
        #print x.element, x.id, y.id, y.element
        #print
        break
        
b = fake_statmech()
tor = [bond.atom1.id, bond.atom2.id]
print tor
b.set_rotor_info(xmol, tor)
label = "VMPUAIZSESMILD-UHFFFAOYSA-N"
b.write_arkane_for_reacts_and_prods(aspec, label)

thermoClass = 'NASA'
rmg_spec = RMG_Species()
rmg_spec.fromSMILES(SML)

#ark_spec = arkane.ArkaneSpecies(species=rmg_spec)
ark_therm = arkane.ThermoJob(rmg_spec, thermoClass)
ark_therm.arkane_species.use_hindered_rotors = True

[6, 3]


AttributeError: 'list' object has no attribute 'atom_indices'

In [ ]:
import os
thermo_file = 'thermo_input.py'

modelChemistry = 'M06-2X/cc-pVTZ'

label = 'VMPUAIZSESMILD-UHFFFAOYSA-N'
spec_file = label + '.py'
species = 'geofreq_species'

write_arkThermoInput(thermo_file, modelChemistry, species, spec_file)

In [ ]:
def write_arkThermoInput(filename, modelChemistry, spec_name, spec_file, path=None):
    output = ['#!/usr/bin/env python',
              '# -*- coding: utf-8 -*-',
              '',
              'modelChemistry = "{0}"'.format(modelChemistry),
              'useHinderedRotors = True',
              'useBondCorrections = True',
              '',
              "species('{0}', '{1}')\n\nstatmech('{0}')".format(species, spec_file),
              "thermo('{0}', '{1}')".format(species, 'NASA')]

    output = '\n'.join(output)
    
    print output
    if path is None:
        path = os.getcwd()
    
    with open(os.path.join(path,filename), 'w') as f:
        f.write(output)
        f.close()
    
    return

In [ ]:
thermo = ark.speciesDict["geofreq_species"].thermo
thermo

In [ ]:
import os
input_file = 'geofreq_input.py'
out_path = os.getcwd()
ark = arkane.Arkane(inputFile=input_file, outputDirectory=out_path)
print ark.inputFile
ark.plot = False
ark.execute()

In [ ]:
import os
input_file = 'thermo_input.py'
out_path = os.getcwd()
ark = arkane.Arkane(inputFile=input_file, outputDirectory=out_path)
print ark.inputFile
ark.plot = False
ark.execute()

In [ ]:
from autotst.species import Conformer
SML = 'CC(CO)(C)OC'
conf = Conformer(SML)conf.ase_molecule

In [ ]:
from autotst.calculators.gaussian import read_gaussian_out
geo_log = "VMPUAIZSESMILD-UHFFFAOYSA-N_Geo.log"
conf.ase_molecule = read_gaussian_out(geo_log)
conf.update_coords()

In [44]:
import os
from ase.calculators.gaussian import Gaussian
from autotst.species import Conformer

SMILES = ['CC(CO)(C)OC']
home = os.getcwd()
path = None
given_steps = 36
given_stpsize = 10.0


#def fake_auto(SMILES, path)
if path is None:
    path = os.getcwd()

for SML in SMILES:
    conf = Conformer(SML)
    augInChI = conf.rmg_molecule.toAugmentedInChIKey()
    
    geoFreqCom = augInChI + '_GeoFreq.com'
    geoFreqLog = augInChI + '_GeoFreq.log'
    
    hind = Hindered_Rotors(conf)
    
    gaus_job = Gaussian()
    
    geoFreqCom_Path = os.path.join(path, geoFreqCom)
    geoFreqLog_Path = os.path.join(path, geoFreqLog)
    
    if not exists_and_complete(geoFreqLog_Path):
        if not os.path.isfile(geoFreqCom_Path):
            gaus_job.label = augInChI + '_GeoFreq'
            gaus_job.parameters['method'] = 'm062x'
            gaus_job.parameters['basis'] = '6-311+g(2df,2p)'
            gaus_job.extra = 'opt=(calcfc,maxcycle=1000) freq iop(7/33=1)'
            del gaus_job.parameters['force']

            # Write geo input file
            os.chdir(geoFreqCom_Path)
            #gaus_job.write_input(conf.ase_molecule)
            os.chdir(home)
        
        assert os.path.isfile(geoFreqCom_Path)
        os.chdir(path)
        #subprocess.call(shlex.split('sbatch rotors_run_template.sh {0}'.format(geoFreqLog)))
        os.chdir(home)
        
    else:
        assert hind.update_Conformer(filename=geoFreqLog, path=path)
        hind.generate_Scans(path=path)
        
        allScansUpdated = True
        for scan in hind.scans:
            scan.set_DefaultFiles()
            if exists_and_complete(os.path.join(scan.path, scan.output_log)):
                scan.set_data()
                #TODO scan handling
            else:
                allScansUpdated = False
                if not os.path.isfile(os.path.join(path, scan.input_com)):
                    scan.write_TorInput(steps=given_steps, stepsize_deg=given_stpsize)
                
                os.chdir(path)
                #subprocess.call(shlex.split('sbatch rotors_run_template.sh {0}'.format(scan.input_com)))
                os.chdir(home)
            
        if allScansUpdated:
            statmech_job = fake_statmech()
            statmech_job.model_chemistry = 'M06-2X/cc-pVTZ'
            
            species_py = augInChI + '.py'
            if not os.path.isfile(os.path.join(path, species_py)):
                for torsion in hind.get_Torsions():
                    b.add_rotor(conf, torsion)
                
                statmech_job.write_arkane_for_reacts_and_prods(conf, augInChI)
            
            
            
            
            """
            b = fake_statmech()
            tor = [bond.atom1.id, bond.atom2.id]
            print tor
            b.set_rotor_info(xmol, tor)
            label = "VMPUAIZSESMILD-UHFFFAOYSA-N"
            b.write_arkane_for_reacts_and_prods(aspec, label)

            thermoClass = 'NASA'
            rmg_spec = RMG_Species()
            rmg_spec.fromSMILES(SML)

            #ark_spec = arkane.ArkaneSpecies(species=rmg_spec)
            ark_therm = arkane.ThermoJob(rmg_spec, thermoClass)
            ark_therm.arkane_species.use_hindered_rotors = True
            """

    
os.chdir(home)

In [5]:
from autotst.calculators.gaussian import Gaussian as Auto_Gaus

def exists_and_complete(path):
    exists_and_complete = False
    auto_g = Auto_Gaus()
    
    if os.path.isfile(path):
        if False not in auto_g.verify_output_file(path):
            exists_and_complete = True
    
    return exists_and_complete

In [43]:
import arkane
import cclib

from autotst.calculators.gaussian import read_gaussian_out

from rmgpy.molecule import Molecule as RMG_Molecule

from autotst.species import Species as TS_Species
from autotst.species import Conformer

import matplotlib.pyplot as plt
import numpy as np

class Hindered_Rotors:
    
    def __init__(self, conformer, label=None, path=None):
        """
        Class for hindered rotor calculation methods
        
        conformer :: autotst.species Conformer object
        label :: Automatically assigned if None, used as default prefix to filenames if filename not specificed
        path :: General path where files relevant to this conformer can be found or written to
        
        """
        
        
        self.__conformer__ = conformer
        self.SMILES = conformer.smiles

        self.path = path
        
        
        self.AugInChIKey = conformer.rmg_molecule.toAugmentedInChIKey()
        if label is None:
            #label = self.AugInchiKey.strip('-N')
            label = self.AugInChIKey
        self.label = label
        
        self.__geoIsOpt__ = False
        self.__torsions__ = None

        self.scans = []

        return
    
    def get_Conformer(self):
        return self.__conformer__
    
    def get_GeoIsOpt(self):
        return self.__geoIsOpt__
    
    def get_Torsions(self):
        return self.__torsions__
    
    def update_Conformer(self, filename=None, path=None):
        self.__geoIsOpt__ = False
        self.__torsions__ = None
        
        if filename is None:
            filename = self.label + '_GeoFreq.log'
        
        home = os.getcwd()
        if path is None:
            path = home
            
        os.chdir(path)
        
        self.__conformer__.ase_molecule = read_gaussian_out(filename)
        self.__conformer__.update_coords()
        self.__torsions__ = self.__conformer__.get_torsions()
        self.__geoIsOpt__ = True
        
        os.chdir(home)
        return self.get_GeoIsOpt()
 
    def create_Scan(self, torsion, steps=None, stepsize_deg=None, path=None, label=None):
        """
        Creates scan object and adds it to rotors class
        
        torsion             :: Torsion object
        steps               :: Number of steps taken during scan (int)
        stepsize            :: DEGREES between steps (float)
        path                :: General path where things like geometery logs, scan input, and scan log can be found
        """
        if steps is not None:
            assert isinstance(steps, int)
        if stepsize_deg is not None:
            assert isinstance(stepsize_deg, float)
        
        if self.get_GeoIsOpt():
            conf = self.get_Conformer()
            scan_inst = Scan(conf, torsion, steps, stepsize_deg, path=path, label=None)
            self.scans.append(scan_inst)
        return
    
    
    def generate_Scans(self, steps=None, stepsize_deg=None,path=None, label=None):
        if self.get_GeoIsOpt():
            for torsion in self.get_Torsions():
                self.create_Scan(torsion, steps=steps, stepsize_deg=stepsize_deg, path=path, label=label)
        return
    
    def set_ScanData(self):
        if self.get_GeoIsOpt(): 
            for scan in self.scans:
                scan.set_Data()

        return
    
    def check_AllScanSCFEnergies(self):
        scan_mins = {}
        for scan in self.scans:
            scan_mins[scan] = scan.check_scfEnergyMinIdx() 
        return scan_mins

In [42]:
class Scan:
    
    def __init__(self, 
                 conformer, 
                 torsion, 
                 steps=None, 
                 stepsize_deg=None, 
                 path=None, 
                 label=None, 
                 geo_log=None, 
                 tor_com=None, 
                 tor_log=None):
        
        self.__conformer__ = conformer
        self.__torsion__ = torsion
        
        self.AugInChIKey = conformer.rmg_molecule.toAugmentedInChIKey()
        if label is None:
            #label = self.AugInChIKey.strip('-N')
            label = self.AugInChIKey
        self.label = label
        
        self.path = path
        
        self.i = torsion.atom_indices[0]
        self.j = torsion.atom_indices[1]
        self.k = torsion.atom_indices[2]
        self.l = torsion.atom_indices[3]
        
        if steps is not None:
            assert isinstance(steps, int)
        if stepsize_deg is not None:
            assert isinstance(stepsize_deg, float)
        
        self.steps = steps
        self.stepsize_deg = stepsize_deg
        
        self.stepsize_rad = None
        
            
        self.geo_log = geo_log
        self.input_com = tor_com
        self.output_log = tor_log
        
        if None not in (geo_log, tor_com, tor_log):
            self.set_default_files()
            
        #After running a scan, populate data
        self.data = None
        
        #Data from cclib
        self.opt_indices = None
        self.start_indices = None
        self.opt_scfEnergies = None
        self.atomCoords = None
        
        #Redundant data from Arkane
        self.ark_energies = None
        self.ark_thetas = None
        return
    
    def get_Conformer(self):
        return self.__conformer__
    
    def get_Torsion(self):
        return self.__torsion__
        
    def set_DefaultFiles(self):
        """
        Sets geo_log, input_com, and output_log per naming convention around label
            If label is None in constructor (by default), label is related to AugInchiKey per constructor
        """
        self.geo_log = self.label + '_Geo.log'

        a = min([self.j, self.k])
        b = max([self.j, self.k])
        
        self.input_com = self.label + '_tor{}{}'.format(a, b) + '.com'
        self.output_log = self.label + '_tor{}{}'.format(a, b) + '.log'
        return
    
    def write_TorInput(self,
                       filename=None,
                       path=None, 
                       options=None, 
                       torsion=None, 
                       steps=None, 
                       stepsize_deg=None):
        """
        Given filename and options, and taking RMG_Molecule object from scan
        Write input file for torsion of scan
        """
        
        if filename is None:
            assert self.input_com is not None
            filename = self.input_com
        
        if path is None:
            if self.path is None:
                path = os.getcwd()
            else:
                path = self.path
        
        if torsion is None:
            torsion = self.get_Torsion()
        
        if steps is None:
            steps = self.steps
        
        if stepsize_deg is None:
            stepsize_deg = self.stepsize_deg
        
        conf = self.get_Conformer()
        mol = conf.rmg_molecule
        
        mol.updateMultiplicity()
        
        if options is None:
            options = ['%nprocshared=20',
                       '%mem=5GB',
                       '#p m062x/6-311+g(2df,2p) Opt=(CalcFC,ModRedun)']
        
        output = '\n'.join(options)
        output += '\n'.join(['','','0 {}'.format(mol.multiplicity),''])
        
        #Gaussian indexing begins with 1 NOT zero
        for atom in mol.atoms:
            output = output + "{}     {}     {}     {}\n".format(atom.element, atom.coords[0], atom.coords[1], atom.coords[2])

        output = output + '\n'

        for bond in mol.getAllEdges():
            output = output + 'B {0} {1}\n'.format(bond.atom1.id, bond.atom2.id)
        
        output = output + 'D {0} {1} {2} {3} S {4} {5}'.format(self.i+1,
                                                               self.j+1,
                                                               self.k+1,
                                                               self.l+1,
                                                               steps,
                                                               stepsize_deg)
        output = output + '\n\n\n'
        
        with open(os.path.join(path, filename), 'w') as F:
            F.write(output)
            F.close
        
        return
    
    def set_DataArkane(self, scan_log=None):
        if scan_log is None:
            assert self.output_log is not None
            scan_log = self.output_log
        
        ark = arkane.gaussian.GaussianLog(scan_log)
        scan_energy = ark.loadScanEnergies()
        
        self.ark_energies = scan_energy[0]
        self.ark_thetas = scan_energy[1]
        return 
        
    def set_Data(self, scan_log=None):
        #Populating scan data attributes from gaussian scan output log
        if scan_log is None:
            assert self.output_log is not None
            scan_log = self.output_log
        
        self.data = cclib.io.ccread(scan_log)
        
        self.set_data_Arkane(scan_log=scan_log)
        
        self.opt_indices = [i for i, status in enumerate(self.data.optstatus) if status==2]
        self.start_indices = [i for i, status in enumerate(self.data.optstatus) if status==1]
        self.opt_scfEnergies = [self.data.scfenergies[index] for index in self.opt_indices]
        
        assert len(self.opt_scfEnergies) == len(self.ark_energies)
        
        if self.steps is None:
            self.steps = len(self.opt_scfEnergies) - 1
        else:
            assert self.steps == len(self.opt_scfEnergies) - 1
        
        #Getting stepsize via Arkane and comparing it with the given stepsize
        self.stepsize_rad = self.ark_thetas[1] - self.ark_thetas[0]
        
        if self.stepsize is not None:
            assert abs(self.stepsize_deg/360 - self.stepsize_rad/(2*3.1415)) < 0.01
        
        assert len(self.opt_indices) == self.steps + 1
        assert len(self.start_indices) == self.steps + 1

        """
        #Keeping atomcoords organized with their respective atom nos for easy ID
        atom_id_coords = []
        atomids = self.data.atomnos
        for i, geo_coords in enumerate(self.data.atomcoords):
            #Setting up [atomno, x, y, x] for each atom in a geometry
            geo_id_coords = np.insert(geo_coords, 0, atomids, axis=1)
            #Adding all of individual geometries to master list
            atom_id_coords.append(geo_id_coords)
        
        self.atomcoords = atom_id_coords"""
        
        return True
    
    def plot_Scan(self):
        plt.plot(self.ark_thetas, self.ark_energies)
        return
    
    
    def check_ArkThetaContinuous(self, tol=None):
        """
        Returns true if ark_energies at the same theta are within tolerance
        
        """
        if tol is None:
            tol = 10**-8
        
        energy_by_theta = {}
        
        for theta, energy in zip(self.ark_thetas, self.ark_energies):
            key = int(theta*100/(2*3.1415))
            
            if key in energy_by_theta.keys():
                previous = energy_by_theta[key]
                error = 1.00000000-previous/energy
                
                if abs(error) > tol:
                    return False
            else:
                energy_by_theta[key] = energy
            
        return True
    
    
    def check_ArkSlopeContinuous(self, tol=None):
        """
        Check if change in energy within given tolerance over ark energies
        
        """
        if tol is None:
            tol = 10**10
        
        for i in range(1, len(self.opt_scfEnergies)):
            
            slope = (self.ark_energies[i]-self.ark_energies[i-1]) / self.stepsize_rad
            
            if abs(slope)>tol:
                return False
        
        return True
    
    def check_SCFSlopeContinuous(self, tol=None):
        """
        Check if change in energy within given tolerance over opt scf energies
        
        """
        if tol is None:
            tol = 10**-2.5
        
        for i in range(1, len(self.opt_scfEnergies)):
            
            slope = (self.opt_scfEnergies[i]-self.opt_scfEnergies[i-1]) / self.stepsize_deg
            
            if abs(slope)>tol:
                return False
        return True
    
    
    def get_ArkEnergyMinIdx(self):
        """
        Returns index of minimum ark_energy
        
        """
        opt_min_idx = 0
        min_en = self.ark_energies[opt_min_idx]
        
        for i, energy in enumerate(self.ark_energies):
            if energy < min_en:
                min_en = energy
                opt_min_idx = i
        
        return opt_min_idx
    
    def get_SCFEnergyMinIdx(self):
        """
        Returns list of index of min energy among opt energies and among all energies found
        """
        opt_min_idx = 0
        min_en = self.opt_scfEnergies[opt_min_idx]
        min_idx = self.opt_indices[opt_min_idx]
        
        for i, energy in enumerate(self.ark_energies):
            if energy < min_en:
                min_en = energy
                opt_min_idx = i
                min_idx = self.opt_indices[i]
        
        return [opt_min_idx, min_idx]
    

In [90]:
#hin_inst = hindered_rotors('CCCC(=O)OC')
hin_inst = hindered_rotors('CC(CO)(C)OC', default=False)
hin_inst.updateMol()
hin_inst.set_torsions()
hin_inst.set_scans(36, 10.0)

for tor_scan in hin_inst.scans:
    tor_scan.get_data()
    print tor_scan.j, tor_scan.k
    print tor_scan.geo_log
    print tor_scan.input_com
    print tor_scan.output_log
    break

7 1
VMPUAIZSESMILD-UHFFFAOYSA-N_Geo.log
VMPUAIZSESMILD-UHFFFAOYSA-N_tor17.com
VMPUAIZSESMILD-UHFFFAOYSA-N_tor17.log


In [91]:
hin_inst.updateMol()
freq_file =hin_inst.write_Freq_Input()

In [60]:
x = Conformer('CC(CO)(C)OC')

In [ ]:
data=tor_scan.data
steps = 36
theta = 10.0

opt_indices = [i for i, status in enumerate(data.optstatus) if status==2]
opt_energies = [data.scfenergies[index] for index in opt_indices]
len(opt_energies)

thetas = [i*theta for i in range(steps+1)]
plt.plot(thetas, opt_energies)
#plt.plot(opt_energies)
plt.show()

In [93]:
print freq_file
base_file = freq_file[:-9]
print base_file

VMPUAIZSESMILD-UHFFFAOYSA-N_Freq.com
VMPUAIZSESMILD-UHFFFAOYSA-N


In [94]:
import subprocess, shlex
base_filename = 'VMPUAIZSESMILD-UHFFFAOYSA-N_Freq'
subprocess.call(shlex.split('sbatch rotors_run_template.sh {0}'.format(base_filename)))

0

In [ ]:
# To create new template run-script

#rotors_runscript_template = "#!/bin/bash\n\n#SBATCH --job-name=$1\n#SBATCH --output=$1.log\n\n## number of nodes\n#SBATCH -N 1\n#SBATCH --exclusive\n#SBATCH --partition=general\n#SBATCH --mem=120000\n\n## set the gaussian scratch directory to a fast drive\n## note that /tmp/ may be even faster than /gss_gpfs_scratch/\n#export GAUSS_SCRDIR=/scratch/$USER/gaussian_scratch\n## make the directory if it doesn't exist already\n#mkdir -p $GAUSS_SCRDIR\n\n# run gaussian, with the desired input file\ng16 $1.com\n\n"

lst_template = ["#!/bin/bash",
                "",
                "#SBATCH --job-name=rotor",
                "#SBATCH --output=$1.log",
                "",
                "## number of nodes",
                "#SBATCH -N 1",
                "#SBATCH --exclusive",
                "#SBATCH --partition=general",
                "#SBATCH --mem=120000",
                "",
                "## export GAUSS_SCRDIR=/scratch/$USER/gaussian_scratch",
                "## make the directory if it doesn't exist already",
                "## mkdir -p $GAUSS_SCRDIR",
                "",
                "# run gaussian, with the desired input file",
                "g16 $1.com",
                "",
                ""]

#new_script_template = open('rotors_run_template.sh', 'w')
#new_script_template.write('\n'.join(lst_template))
print '\n'.join(lst_template)
#new_script_template.close()
#print rotors_runscript_template


In [ ]:
# Writing Geo opt input as AutoTST would

from autotst.species import Conformer
from ase.calculators.gaussian import *
test_mol = Conformer('CCCC(=O)OC')

# Preparing gaussian input file parameters for Geo Opt as AutoTST does it
test_g = Gaussian()
test_g.label = test_mol.rmg_molecule.toAugmentedInChIKey() + '_Geo'
print test_g.label + '.com'
print
test_g.parameters['method'] = 'm062x'
test_g.parameters['basis'] = '6-311+g(2df,2p)'
test_g.extra = 'opt=(calcfc,maxcycle=1000)'
del test_g.parameters['force']

# Checking parameters are good
for key, val in test_g.parameters.items():
    print key, val

# Write geo input file
#test_g.write_input(test_mol.ase_molecule)
test_mol.view()

In [41]:
# Writing Freq input as AutoTST would

from autotst.species import Conformer
from ase.calculators.gaussian import *
test_mol = Conformer('CC(CO)(C)OC')

# Preparing gaussian input file parameters for Geo Opt as AutoTST does it
test_gf = Gaussian()
test_gf.label = test_mol.rmg_molecule.toAugmentedInChIKey() + '_Freq'
print test_gf.label + '.com'
print
test_gf.parameters['method'] = 'm062x'
test_gf.parameters['basis'] = '6-311+g(2df,2p)'
test_gf.extra = 'freq iop(7/33=1)'
del test_gf.parameters['force']
test_gf.atoms

# Checking parameters are good
for key, val in test_gf.parameters.items():
    print key, val

# Write geo input file
#test_g.write_input(test_mol.ase_molecule)
#test_mol.view()

VMPUAIZSESMILD-UHFFFAOYSA-N_Freq.com

charge 0
method m062x
basis 6-311+g(2df,2p)


In [ ]:
#!/bin/bash

#SBATCH --job-name={}
#SBATCH --output={}.log

## number of nodes
#SBATCH -N 1
#SBATCH --exclusive
#SBATCH --partition=general
#SBATCH --mem=120000

## set the gaussian scratch directory to a fast drive
## note that /tmp/ may be even faster than /gss_gpfs_scratch/
#export GAUSS_SCRDIR=/gss_gpfs_scratch/$USER/gaussian_scratch
## make the directory if it doesn't exist already
#mkdir -p $GAUSS_SCRDIR

# run gaussian, with the desired input file
g16 {}.com

    
"""    def write_Geo_Input(self):
        # Preparing gaussian input file for Geo Opt
        gaus_job = Gaussian()
        gaus_job.label = self.label + '_Geo'
        gaus_job.parameters['method'] = 'm062x'
        gaus_job.parameters['basis'] = '6-311+g(2df,2p)'
        gaus_job.extra = 'opt=(calcfc,maxcycle=1000)'
        del gaus_job.parameters['force']

        # Write geo input file
        gaus_job.write_input(.ase_molecule)"""
"""    
    def write_Freq_Input(self, freq_file=None,  path=None, options=None):
        # Preparing gaussian input file for Freq Calc
        if self.geoIsOpt:
            mol = self.getRMGMol()
            mol.updateMultiplicity()
            
            if freq_file is None:
                freq_file = self.label + '_Freq.com'
                
            if path is None:
                path = self.path
            
            if options is None:
                options = ['%nprocshared=20',
                           '%mem=5GB',
                           '#p m062x/6-311+g(2df,2p) freq iop(7/33=1)']

            output = '\n'.join(options)
            
            output += '\n'.join(['','','0 {}'.format(mol.multiplicity),''])

            n = 1
            for atom in mol.atoms:
                assert n == atom.id
                n += 1
                output += "{}     {}     {}     {}\n".format(atom.element, atom.coords[0], atom.coords[1], atom.coords[2])
            
            with open(os.path.join(path, freq_file), 'w') as f:
                f.write(output)
                f.close()
            return
    
    def getRMGMol(self):
        return self.__RMGMol__
"""            
"""
    def updateMol(self, geo_file=None, path=None):
        assert self.__RMGMol__ is None, "Highly recommended not to update molecule more than once"
        
        if geo_file is None:
            geo_file = self.label + '_Geo.log'
        if path is None:
            path = self.path
        
        atoms = None
        try:
            atoms = read_gaussian_out(geo_file)
        except:
            print "Failed to Find Valid Geo_file"
            return False
        
        #Update XYZ
        mol = RMG_Molecule()
        mol.fromXYZ(atoms.get_atomic_numbers(), atoms.get_positions())
        
        #Update IDs
        i = 1
        for atom in mol.atoms:
            atom.id = i
            i += 1
        
        #Update Multiplicity
        mol.updateMultiplicity()
        
        self.__RMGMol__ = mol
        self.geoIsOpt = True
        return True
    
    
            
    def set_torsions(self):
        Method setting torsions of simple chains to hindered rotors object

        NO RINGS
        NO RINGS
        NO RINGS

        A - Bond1 - B - Bond2 - C - Bond3 - D

        i = A.index
        j = B.index
        k = C.index
        l = D.index

        Torsion is unique if j,k & k,j pairing is unique

        """
"""
        if not self.geoIsOpt:
            self.updateMol()
        
        assert self.geoIsOpt
        mol = self.getRMGMol()
 
        # dict of (j, k) key with [i,j,k,l] value
        # Wipes existing torsions away to recalculate
        self.torsions = {}
            
        for bond2 in mol.getAllEdges():
            i, j, k, l = -1,-1,-1,-1
            B = bond2.atom1
            j = bond2.atom1.id

            C = bond2.atom2
            k = bond2.atom2.id

            assert (j,k) not in self.torsions.keys() or (k,j) in torsions.keys()
            # [i,j,k,l] and [i,j,k,m] torsion is the same to Gaussian as long as center is k&j or j&k
            #continue

            found_bond1 = False
            found_bond3 = False

            for Bbond in mol.getBonds(B).values():
                if (Bbond.atom1.id == B.id) and (Bbond.atom2.id != C.id):
                    #Bbond is not bond2
                    #Bbond must be bond1
                    found_bond1 = True
                    i = Bbond.atom2.id
                elif (Bbond.atom1.id != C.id) and (Bbond.atom2.id == B.id):
                    #Bbond is not bond2
                    #Bbond must be bond1
                    found_bond1 = True
                    i = Bbond.atom1.id
                else:
                    #Bbond is bond2
                    assert (Bbond.atom1 == C) or (Bbond.atom2 == C)

            for Cbond in mol.getBonds(C).values():
                if (Cbond.atom1.id == C.id) and (Cbond.atom2.id != B.id):
                    #Bbond is not bond2
                    #Bbond must be bond3
                    found_bond3 = True
                    l = Cbond.atom2.id
                elif (Cbond.atom1.id != B.id) and (Cbond.atom2.id == C.id):
                    #Bbond is not bond2
                    #Bbond must be bond1
                    found_bond3 = True
                    l = Cbond.atom1.id
                else:
                    #Cbond is bond2
                    assert (Cbond.atom1 == B) or (Cbond.atom2 == B)

            #print found_bond1, found_bond3
            #print i, j, k, l
            #print
            if found_bond1 and found_bond3:
                self.torsions[(j,k)] = [i,j,k,l]
        return
"""   
"""
    def Rote_write_geo_com(self, filename=None):
        
        #Writes Gaussian input file for a geometry optimization of self.__RMGMol__
        #with a basisand parameters similar to AutoTST
        
        auto_mol = None
        if filename is None:
            filename = self.AugInchiKey + '_Geo.com'

        # Preparing gaussian input file parameters for Geo Opt as AutoTST does it
        g_inst = Gaussian()
        g_inst.label = self.AugInchiKey + '_Geo'
        
        g_inst.parameters['method'] = 'm062x'
        g_inst.parameters['basis'] = '6-311+g(2df,2p)'
        g_inst.extra = 'opt=(calcfc,maxcycle=1000)'
        del g_inst.parameters['force']

        # Write Gaussian input file
        # TODO Fix
        test_g.write_input(auto_mol.ase_molecule)
        return False
    """
